# Single Modality Analysis of Mouse Spleen Dataset

## Loading packages

In [ ]:
import warnings
warnings.filterwarnings('ignore')
## Loading package
import os

import scanpy as sc
import matplotlib.pyplot as plt

# the location of R (used for the mclust clustering)
os.environ['R_HOME'] = 'E:/R-4.3.1'
os.environ['R_USER'] = 'E:/anaconda/lib/site-packages/rpy2'

save_path = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Visualization/Mouse_Spleen/' 

import sys
sys.path.insert(0, 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Model/')
from preprocess import preprocessing
from utils import mclust_R

## Loading and Preprocessing data

In [ ]:
file_fold_1 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Spleen_1/'
file_fold_2 = 'D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/data/Mouse_Spleen_2/'

adata_omics_1_1 = sc.read_h5ad(file_fold_1 + 'adata_RNA.h5ad')
adata_omics_1_2 = sc.read_h5ad(file_fold_1 + 'adata_ADT.h5ad')

adata_omics_2_1 = sc.read_h5ad(file_fold_2 + 'adata_RNA.h5ad')
adata_omics_2_2 = sc.read_h5ad(file_fold_2 + 'adata_ADT.h5ad')

adata_omics_1_1.var_names_make_unique()
adata_omics_1_2.var_names_make_unique()
adata_omics_2_1.var_names_make_unique()
adata_omics_2_2.var_names_make_unique()

adata_omics_1_1, adata_omics_1_2 = preprocessing(adata_omics_1_1, adata_omics_1_2, 'SPOTS')
adata_omics_2_1, adata_omics_2_2 = preprocessing(adata_omics_2_1, adata_omics_2_2, 'SPOTS')

SPOTS data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (2568, 3000)
Dimensions after preprocessing adata_modal_2: (2568, 21)
SPOTS data preprocessing have done!
Dimensions after preprocessed adata_modal_1: (2768, 3000)
Dimensions after preprocessing adata_modal_2: (2768, 21)


In [ ]:
adata_RNA_analysis = adata_omics_2_1
adata_Protein_analysis = adata_omics_2_2

In [ ]:
## Running PCA on RNA modality
sc.pp.pca(adata_RNA_analysis, n_comps=adata_Protein_analysis.shape[1]-1)
sc.pp.neighbors(adata_RNA_analysis, use_rep='X_pca')
sc.tl.umap(adata_RNA_analysis)
mclust_R(adata_RNA_analysis, used_obsm='X_pca', num_cluster=6)

## Running PCA on Protein modality
sc.pp.pca(adata_Protein_analysis, n_comps=adata_Protein_analysis.shape[1]-1)
sc.pp.neighbors(adata_Protein_analysis, use_rep='X_pca')
sc.tl.umap(adata_Protein_analysis)
mclust_R(adata_Protein_analysis, used_obsm='X_pca', num_cluster=6)

fitting ...
  |======================================================================| 100%
fitting ...
  |======================================================================| 100%


AnnData object with n_obs × n_vars = 2768 × 21
    obs: 'clusters_mclust'
    var: 'mean', 'std'
    uns: 'INR', 'pca', 'neighbors', 'umap'
    obsm: 'spatial', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

## Storing the results

In [27]:
results = sc.read_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Mouse_Spleen_Replicate2.h5ad')
results

AnnData object with n_obs × n_vars = 2768 × 0
    obs: 'SpaKnit', 'SpatialGlue', 'STAGATE', 'SpaGCN', 'Modality1', 'Modality2'
    uns: 'SpaKnit_colors'
    obsm: 'Modality1', 'Modality2', 'STAGATE', 'SpaKnit', 'SpatialGlue', 'spatial'

In [22]:
results.obs['Modality1'] = adata_RNA_analysis.obs['clusters_mclust'].values
results.obsm['Modality1'] = adata_RNA_analysis.obsm['X_pca']
results.obs['Modality2'] = adata_Protein_analysis.obs['clusters_mclust'].values
results.obsm['Modality2'] = adata_Protein_analysis.obsm['X_pca']
results.write_h5ad('D:/study/learning/spatial_transcriptome/papers/spatial_multi_omics-main/Results/Mouse_Spleen_Replicate2.h5ad')

In [ ]:
colors = [
    '#fdf0d5', '#f9c74f', '#83c5be', '#99582a', '#3f5e66', '#ee6055'
]

fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
sc.pl.embedding(adata_Protein_analysis, basis='spatial', color='clusters_mclust', ax=ax, s=180, show=False, palette=colors)
ax.set_title(f'')
ax.set_xlabel('')
ax.set_ylabel('')
# remove legend
ax.get_legend().remove()
for spine in ax.spines.values():
    spine.set_visible(False)
plt.tight_layout()

In [ ]:
## RNA UMAP
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.pl.umap(adata_RNA_analysis, color='clusters_mclust', ax=ax,legend_loc='on data',legend_fontoutline=5, show=False)
ax.set_title('UMAP visulization')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
# plt.savefig(save_path + 'Spleen2_RNA_UMAP.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'Spleen2_RNA_UMAP.eps', bbox_inches='tight', dpi=500)

In [ ]:
# # RNA PAGA
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.tl.paga(adata_RNA_analysis, groups='clusters_mclust')
sc.pl.paga(adata_RNA_analysis, edge_width_scale=1, node_size_scale=5, ax=ax, show=False, threshold=0.1, fontoutline=3)
ax.set_title('PAGA graph')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')

# plt.savefig(save_path + 'Spleen2_RNA_PAGA.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'Spleen2_RNA_PAGA.eps', bbox_inches='tight', dpi=500)

In [ ]:
## Protein clustering
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))

colors_PCA_Protein = ['#8ac926', '#1982c4', '#6a4c93', '#ff595e', '#ffca3a']
sc.pl.embedding(adata_Protein_analysis, basis='spatial', color='clusters_mclust', ax=ax, s=110, palette=colors_PCA_Protein, show=False)
ax.set_title('Protein analysis')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
# remove legend
ax.get_legend().remove()


# plt.savefig(save_path + 'Spleen2_Protein.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'Spleen2_Protein.eps', bbox_inches='tight', dpi=500)

In [ ]:
## Protein UMAP
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.pl.umap(adata_Protein_analysis, color='clusters_mclust', ax=ax,legend_loc='on data',legend_fontoutline=5, show=False)
ax.set_title('UMAP visulization')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')
plt.savefig(save_path + 'Spleen2_Protein_UMAP.png', bbox_inches='tight', dpi=500)
plt.savefig(save_path + 'Spleen2_Protein_UMAP.eps', bbox_inches='tight', dpi=500)

In [ ]:
# # Protein PAGA
plt.rcParams['font.size'] = 20
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(6,6))
sc.tl.paga(adata_Protein_analysis, groups='clusters_mclust')
sc.pl.paga(adata_Protein_analysis, edge_width_scale=1, node_size_scale=5, ax=ax, show=False, threshold=0.1, fontoutline=3)
ax.set_title('PAGA graph')
# remove x, y axis
ax.set_xlabel('')
ax.set_ylabel('')

plt.savefig(save_path + 'Spleen2_Protein_PAGA.png', bbox_inches='tight', dpi=500)
plt.savefig(save_path + 'Spleen2_Protein_PAGA.eps', bbox_inches='tight', dpi=500)

In [ ]:
## sc.tl.rank_genes_groups：Genes are ranked to determine population characteristics
sc.tl.rank_genes_groups(adata_RNA_analysis, 'clusters_mclust', method="t-test")
rank_genes = sc.get.rank_genes_groups_df(adata_RNA_analysis, group=None)
rank_genes.to_excel(save_path + 'marker_genes.xlsx', index=True)

In [ ]:
plt.rcParams['font.size'] = 18
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,5, figsize=(20,4))
fig.subplots_adjust(wspace=0.1, hspace=0)

marker_genes = ['Hba-a2', 'mt-Co2', 'Hba-a2', 'mt-Co2', 'Marco']
components_range = range(5)
for i in components_range:
    sc.pl.embedding(adata_RNA_analysis, basis='spatial', color=marker_genes[i], ax=ax[i], title=marker_genes[i], s=50, colorbar_loc=None, show=False, cmap='coolwarm')
    # remove x, y axis
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')
# plt.savefig(save_path + 'Spleen2_Marker.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'Spleen2_Marker.eps', bbox_inches='tight', dpi=500)
plt.show()

In [ ]:
plt.rcParams['font.size'] = 16
plt.rcParams['font.sans-serif'] = 'Arial'
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax = sc.pl.stacked_violin(adata_RNA_analysis, marker_genes, groupby='clusters_mclust', figsize=(15,8), ax=ax,dendrogram=True, show=False)
# plt.savefig(save_path + 'spleen2_Marker_violin.png', bbox_inches='tight', dpi=500)
# plt.savefig(save_path + 'spleen2_Marker_violin.eps', bbox_inches='tight', dpi=500)
plt.show()